In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime, timedelta
import subprocess

In [3]:
# Initialize SparkSession
spark = SparkSession.builder.appName("BusinessQueries").enableHiveSupport().getOrCreate()

In [4]:
spark.sql("USE iti_dwh")

""


In [5]:
#most selling products
spark.sql(f"""
    SELECT 
        p.product_id, 
        p.product_name, 
        SUM(s.units) AS total_units_sold
    FROM 
        sales_transactions_fact s
    JOIN 
        product_dim p 
    ON 
        s.product_id = p.product_id
    GROUP BY 
        p.product_id, p.product_name
    ORDER BY 
        total_units_sold DESC
""")


product_id,product_name,total_units_sold
22,Coffee Maker,379
25,Washing Machine,365
9,Boots,337
7,Dress,336
27,Iron,322
17,Blouse,319
28,Hair Dryer,308
1,Laptop,306
24,Blender,305
6,Jeans,302


In [5]:
#lowest city in online sales
spark.sql(f"""
    SELECT 
        c.city, 
        SUM(s.total_paid_price) AS total_online_sales
    FROM 
        sales_transactions_fact s
    JOIN 
        customer_dim c 
    ON 
        s.customer_id = c.customer_id
    where is_online = true
    GROUP BY 
        c.city
    ORDER BY 
        total_online_sales ASC
""")


city,total_online_sales
Port Charlotte,151.962
Colchester,151.962
Scituate,299.99
Lawrence,299.99
San Luis Obispo,323.4805
Concord,339.83
Mexico Beach,569.905
Duxbury,573.809
Dummerston,573.809
Richmond,925.8910000000001


In [6]:
#most redeemed offers
spark.sql(f"""
    SELECT 
        discount_percentage, 
        COUNT(*) AS number_of_sales
    FROM 
        sales_transactions_fact
    GROUP BY 
        discount_percentage
    ORDER BY 
        number_of_sales DESC
""")

discount_percentage,number_of_sales
0.0,747
20.0,168
15.0,162
10.0,151
5.0,138
25.0,134


In [8]:
#most redeemed offers per product
spark.sql(f"""
    SELECT 
        p.product_id, 
        p.product_name, 
        s.discount_percentage, 
        COUNT(*) AS purchase_count
    FROM 
        sales_transactions_fact s
    JOIN 
        product_dim p 
    ON 
        s.product_id = p.product_id
    WHERE 
        s.discount_percentage <> 0
    GROUP BY 
        p.product_id, p.product_name, s.discount_percentage
     ORDER BY 
        purchase_count DESC
""")


product_id,product_name,discount_percentage,purchase_count
4,Headphones,10.0,20
11,TV,20.0,19
24,Blender,10.0,19
26,Vacuum Cleaner,15.0,19
2,Smartphone,20.0,18
19,Sandals,20.0,17
14,Camera,15.0,17
22,Coffee Maker,15.0,16
29,Hair Straightener,20.0,15
8,Sneakers,20.0,15


In [9]:
spark.stop()